## Lab. 12

### Solve the following problem using Genetic Algorithms:


Problem: Weighted N-Queen Problem


You are given an N×N chessboard, and each cell of the board has an associated weight. Your task is to find a valid placement of N queens such that the total weight of the queens is maximized, and no two queens threaten each other.





In the traditional N-Queen Problem, the goal is to place N queens on an N×N chessboard in such a way that no two queens threaten each other. In this variation, we introduce weights to the queens and aim to find a placement that maximizes the total weight of the queens while satisfying the constraint of non-threatening positions.


Constraints:

1. There should be exactly one queen in each row and each column.
2. No two queens should be placed in the same diagonal, i.e., they should not threaten each other.
3. The placement should maximize the total weight of the queens.


Representation:

Use a permutation-based representation. Each permutation represents the column position of the queen for each row. 

For example, if N=4, a valid permutation [2, 4, 1, 3] indicates that the queen in the first row is placed in column 2, the queen in the second row is placed in column 4, and so on.


Genetic Algorithm Steps:

1. *Initialization*: Generate an initial population of permutations randomly.

2. *Fitness Evaluation*: Evaluate the fitness of each permutation by calculating the total weight of the queens while considering the non-threatening positions.

3. *Selection*: Select a subset of permutations from the population based on their fitness, using selection techniques like tournament selection or roulette wheel selection.

4. *Crossover*: Perform crossover (recombination) on the selected permutations to create new offspring permutations.

5. *Mutation*: Introduce random changes (mutations) in the offspring permutations to maintain diversity in the population.

6. *Fitness Evaluation for the new individuals*: Evaluate the fitness of the new population.

7. *Form the new population*: Select the surviving individuals based on scores, with chances direct proportional with their performance.

8. Repeat steps 3-7 for a certain number of generations or until a termination condition is met (e.g., a maximum number of iterations or a satisfactory solution is found).


9. *Termination*: Return the best-performing individual (permutation) found as the solution to the problem.

Note: The fitness function used in this problem should calculate the total weight of the queens based on the positions specified by the permutation. Additionally, the fitness function should penalize solutions that violate the non-threatening constraint by assigning a lower fitness score to such permutations.

In [9]:
import random

# constants
N = 8  # board size
POPULATION_SIZE = 100  # nr individuals in the population
MAX_GENERATIONS = 100  # max nr generations

# board weights
weights = [[random.randint(1, 10) for _ in range(N)] for _ in range(N)]


def generate_random_permutation():
    """
    Generate random permutation representing the column position of the queens for each row.
    """
    return random.sample(range(N), N)


def calculate_fitness(perm):
    """
    Calculate the fitness of a permutation by calculating the total weight of the queens and penalizing
    bad positions.
    """
    total_weight = 0
    for row in range(N):
        col = perm[row]
        total_weight += weights[row][col]
        for i in range(row + 1, N):
            if col == perm[i] or abs(row - i) == abs(col - perm[i]):
                total_weight -= weights[row][col]  # Penalize bad positions
    return total_weight


def selection(population, k):
    """
    Tournament selection to select k individuals from the population.
    """
    selected = []
    for _ in range(k):
        tournament = random.sample(population, 3)  # Randomly select 3 individuals for tournament
        selected.append(max(tournament, key=lambda x: calculate_fitness(x)))  # Select best individual
    return selected


def crossover(parent1, parent2):
    """
    Crossover between 2 parents to create offspring.
    """
    crossover_points = sorted(random.sample(range(N), 2))  # randomly select 2 crossover points
    start, end = crossover_points

    # create offspring with genes from parent1 in the crossover range
    offspring1 = parent1[start:end]
    # fill remaining genes in offspring1 with genes from parent2
    aux = [gene for gene in parent2 if gene not in offspring1]
    offspring1 = aux[:start] + offspring1 + aux[start:]
    
    # create offspring with genes from parent2 in crossover range
    offspring2 = parent2[start:end]
    aux = [gene for gene in parent1 if gene not in offspring2]
    offspring2 = aux[:start] + offspring2 + aux[start:]
    
    return offspring1, offspring2


def mutation(perm):
    """
    Perform mutation by swapping 2 random positions in the permutation.
    """
    idx1, idx2 = random.sample(range(N), 2)
    perm[idx1], perm[idx2] = perm[idx2], perm[idx1]
    return perm


def genetic_algorithm():
    # init
    population = [generate_random_permutation() for _ in range(POPULATION_SIZE)]

    # evolution loop
    for generation in range(MAX_GENERATIONS):
        # Fitness evaluation
        fitness_scores = [calculate_fitness(perm) for perm in population]

        # termination condition
        if max(fitness_scores) == sum(range(1, N + 1)):  # Solution found
            best_permutation = population[fitness_scores.index(max(fitness_scores))]
            return best_permutation

        # selection
        selected = selection(population, k=POPULATION_SIZE // 2)

        # crossover
        offspring = []
        for i in range(0, len(selected), 2):
            parent1 = selected[i]
            parent2 = selected[i + 1]
            offspring1, offspring2 = crossover(parent1, parent2)
            offspring.extend([offspring1, offspring2])

        # mutation
        offspring = [mutation(perm) for perm in offspring]

        # fitness evaluation for new individuals
        offspring_fitness_scores = [calculate_fitness(perm) for perm in offspring]

        # form the new population
        combined_population = population + offspring
        combined_fitness_scores = fitness_scores + offspring_fitness_scores
        population = [ind for _, ind in sorted(zip(combined_fitness_scores, combined_population), reverse=True)][:POPULATION_SIZE]

    # termination
    best_permutation = population[0]
    return best_permutation


best_solution = genetic_algorithm()
print("Best solution:", best_solution)


Best solution: [6, 3, 1, 7, 5, 0, 2, 4]
